In [8]:
from __future__ import division
from __future__ import print_function

import argparse
import time

import numpy as np
import scipy.sparse as sp
import torch
import torchvision
from torch import optim
import pandas as pd


from gae.model import GCNModelVAE

from gae.optimizer import loss_function
from gae.utils import load_data, mask_test_edges, preprocess_graph, get_roc_score

import import_ipynb
from gaemain import gae_for

from scipy.stats.stats import pearsonr  
from scipy import spatial

In [9]:
# N=7
# N2=(N-1)*10 + N
# image_path="./orl_dataset/person"+str(N)+"/train_images/"+str(N2)+"_"+str(N)+".jpg"
# Z=gae_for(image_path)
# Z=Z.detach().numpy()
# # print(N,N2)

In [10]:
testSets=[]
for person in range(1,10):
    path="./orl_dataset/person"+str(person)+"/test_image/"
    files=os.listdir(path)
    count=0
    print("images per person=",len(files))
    for image in files:
        image_path=path+image
        print(image_path)
        count=count+1
        Z=gae_for(image_path)
        # Z=Z.detach().numpy() #convert to Numpy array
        Z = pd.DataFrame(Z) #convert to a dataframe
        testSets.append(Z)
        print("./orl_dataset/person"+str(person)+"/latent_representation_2/testfile.csv")
        # Z.to_csv(csv_path,index=False) #save to file
    print("testing of ",person," is completed")
print(len(testSets))

images per person= 1
./orl_dataset/person1/test_image/1_1.jpg
./orl_dataset/person1/latent_representation_2/testfile.csv
testing of  1  is completed
images per person= 1
./orl_dataset/person2/test_image/12_2.jpg
./orl_dataset/person2/latent_representation_2/testfile.csv
testing of  2  is completed
images per person= 1
./orl_dataset/person3/test_image/30_3.jpg
./orl_dataset/person3/latent_representation_2/testfile.csv
testing of  3  is completed
images per person= 1
./orl_dataset/person4/test_image/34_4.jpg
./orl_dataset/person4/latent_representation_2/testfile.csv
testing of  4  is completed
images per person= 1
./orl_dataset/person5/test_image/48_5.jpg
./orl_dataset/person5/latent_representation_2/testfile.csv
testing of  5  is completed
images per person= 1
./orl_dataset/person6/test_image/54_6.jpg
./orl_dataset/person6/latent_representation_2/testfile.csv
testing of  6  is completed
images per person= 1
./orl_dataset/person7/test_image/68_7.jpg
./orl_dataset/person7/latent_represent

In [11]:
# import glob
# testSets=[]
# ind=0
# for person in range(1,10):
#     tests=glob.glob("./orl_dataset/person"+str(person)+"/latent_representation_2/*.csv")
#     for test in tests: #for eacg file
#         testSets.append(np.loadtxt(test,delimiter=','))  #sign[ind]= content of file, seperated by comma
#         testSets[ind]=testSets[ind][1:]
#         ind+=1
#     print(len(testSets))
# print(tests)
# print(len(testSets))

In [14]:
import numpy as np
import glob
import os

acc=[]
avg_p=[]
avg_recall=[]
avg_fScore=[]
betaSquare=0.5*0.5
r_rate=0 #recognition rate
pr_rate=0 #recognition rate
acc=[]

# files = glob.glob('./orl_dataset/latent1/*.csv')
sign=[]  #15*null

index=0
count=1

for testSet in testSets:
  data = []
  test=[]
  # for file in files:
  TP=0
  TN=0
  FP=0
  FN=0
  tdata=testSet

  # print("Z=",type(Z))
  dist=[]
  pearson_dist=[]
  cosine_dist=[]
  c_dist=[]

  orig_path="./orl_dataset/"
  files=os.listdir(orig_path)
  tests=[]
  # count=0
  for file in files:
    # path=file + '/latent_representation/'
    tests=glob.glob(orig_path + file + '/latent_representation/*.csv')

    sign=[None]*15  #15*null
    ind=0
    # print("files=",len(files))
    for test in tests: #for eacg file
      sign[ind]=np.loadtxt(test,delimiter=',')  #sign[ind]= content of file, seperated by comma
      sign[ind]=sign[ind][1:]
      ind+=1

    # print("test ",tests)
    # print("sign[0]=",sign[0])

    for ele in sign:
      if ele is None:
        break
      dist.append(np.linalg.norm(ele-tdata))
      cosine_dist.append(1 - spatial.distance.cosine(np.array(tdata).flatten() ,np.array(ele).flatten()))
      pearson_dist.append(pearsonr(np.array(tdata).flatten() ,np.array(ele).flatten())[0])
      c_dist.append(pearsonr(np.array(tdata).flatten() ,np.array(ele).flatten())[0])

  # min_ind=dist.index(min(dist))
  print("cosine_dist",cosine_dist)
  # print("pearson_dist",pearson_dist)
  max_ind=pearson_dist.index(max(pearson_dist))
  max_cosine=cosine_dist.index(max(cosine_dist))
  max_c_dist=c_dist.index(max(c_dist))
  print("pearson=",max_ind)
  print("cosine=",max_cosine)
  print("c_dist=",max_c_dist)
  print("matched with=", max_c_dist + 1)
  print("index=", index + 1)
  if(index + 1 != max_c_dist + 1):
    print("\t\t\t\tthis is wrong")
    TP=0
    TN=13
    FP=1
    FP=1
  elif(index +1 == max_c_dist + 1):
    r_rate+=1
    TP=1
    TN=14
    FN=0
    FP=0
  #Measures

  Accuracy = (TP + TN )/ (TP + TN + FP + FN) #(all correct / all)
  Misclassification = (FP + FN )/ (TP + TN + FP + FN) #(all incorrect / all)
  Precision  = TP / (TP + FP) #(true positives / predicted positives)
  deno=TP+FP
  Sensitivity   = TP / (deno) #aka Recall (true positives / all actual positives)
  Specificity  =TN / (TN + FP) #(true negatives / all actual negatives)
  F1_deno=(Precision+Sensitivity)
  if(F1_deno==0):
    F1_Score=0
  else:
    F1_Score=(1+betaSquare)*((Precision*Sensitivity)/(betaSquare*F1_deno))
  # print("\nAccuracy: {}\nMisclassification: {}\nPrecision: {}\nSensitivity: {}\nSpecificity: {}\nF1_Score: {}\n".format(Accuracy,Misclassification,Precision,Sensitivity,Specificity,F1_Score))
  acc.append(Accuracy)
  avg_p.append(Precision)
  avg_recall.append(Sensitivity)
  avg_fScore.append(F1_Score)
  index=index+1
print(r_rate)
print("Total Test Case: {}\nAverage Accuracy: {}\nRecognition Rate: {}\nAverage Precision: {}\nAverage Recall: {}\nAverage F0.5 Score: {}".format(len(acc),(sum(acc)/len(acc)),r_rate/len(acc),(sum(avg_p)/len(avg_p)),(sum(avg_recall)/len(avg_recall)),(sum(avg_fScore)/len(avg_fScore))))

cosine_dist [-0.06255427929428459, 0.029778765571024213, -0.08815222179887328, -0.011654964547035407, 0.12302701171647468, -0.02472380948980768, -0.0561665063152621, 0.022536007491388843, -0.08154644690163804]
pearson= 4
cosine= 4
c_dist= 4
matched with= 5
index= 1
				this is wrong
cosine_dist [0.029025346337658964, 0.012286849303922875, 0.029714551349448892, 0.024317903904728877, 0.04660146424384426, -0.005084473695749825, -0.0017297333673316206, 0.05203042103710076, 0.083601605721912]
pearson= 8
cosine= 8
c_dist= 8
matched with= 9
index= 2
				this is wrong
cosine_dist [0.10597739820406171, -0.06608578770778784, 0.06578509473248118, 0.06558159352895121, -0.02061903172090851, 0.028139198600691362, 0.013438569120516775, -0.013147387880369177, 0.0006440265007963131]
pearson= 0
cosine= 0
c_dist= 0
matched with= 1
index= 3
				this is wrong
cosine_dist [0.05803096261081575, -0.05606029545000468, -0.045030657292853604, 0.016985566594161505, -0.0494914417937673, 0.08852061920328747, 0.0700